In [1]:
import pandas as pd
import time
import tqdm
import json
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options 

In [2]:
df = pd.read_json('parsed_videos.json')
df.head()

,title,link,query
0,Machine Learning Explicado,https://www.youtube.com/watch?v=0PrOA2JK6GQ,https://www.youtube.com/results?search_query=m...
1,O que é Machine Learning? #HipstersPontoTube,https://www.youtube.com/watch?v=Iuz_jc96bQk,https://www.youtube.com/results?search_query=m...
2,Introdução ao Machine Learning (ML de Zero a 1...,https://www.youtube.com/watch?v=t5z5lyrb-7s,https://www.youtube.com/results?search_query=m...
3,O que é Machine Learning (Aprendizado de Máqui...,https://www.youtube.com/watch?v=qDmojpkd8rE,https://www.youtube.com/results?search_query=m...
4,Machine Learning: como ensinar uma máquina a a...,https://www.youtube.com/watch?v=mhe5e2B9bL8,https://www.youtube.com/results?search_query=m...


In [3]:
options = Options()
options.add_argument('--headless')
options.add_argument("--start-maximized")
view_regex = re.compile('([\d]+\s)+')

def convert_str_view(view: str)-> int:
    view = view.replace('.', '')

    match = view_regex.match(view)
    if match is None:
        return 0

    view = match.group(0)

    view = ''.join([s.strip() for s in view if s.strip() != ''])
    return int(view)

resources = []
with webdriver.Chrome(chrome_options=options) as driver:
    for link in tqdm.tqdm_notebook(sorted(df.link.unique())):
        data = dict()

        driver.get(link)
        time.sleep(1)

        data['link'] = link

        try:
            title = driver.find_element_by_css_selector('h1.title.style-scope.ytd-video-primary-info-renderer')
            if title is not None:
                data['title'] = title.text
        except:
            pass

        try:
            canal = driver.find_element_by_css_selector('a.yt-simple-endpoint.style-scope.yt-formatted-string')
            if canal is not None:
                data['canal_nome'] = canal.text
                data['canal_link'] = canal.get_property('href')
        except:
            pass

        try:
            views = driver.find_element_by_css_selector('span.view-count')
            if views is not None:
                data['view_counts'] = convert_str_view(views.text)
        except:
            pass

        try:
            date = driver.find_element_by_css_selector('div#date > yt-formatted-string.style-scope.ytd-video-primary-info-renderer')
            if date is not None:
                data['video_date'] = date.text
        except:
            pass

        try:
            coments = driver.find_elements_by_css_selector('h2#count > yt-formatted-string > span')
            if coments is not None and len(coments) > 0:
                data['coments'] = coments[0].text
        except:
            pass
            
        
        try:
            likes = driver.find_elements_by_css_selector('div#top-level-buttons > ytd-toggle-button-renderer > a > yt-formatted-string')
            if likes is not None and len(likes) == 2:
                data['like'] = likes[0].get_attribute('aria-label')
                data['dislike'] = likes[1].get_attribute('aria-label')
        except:
            pass

        
        resources.append(data)

<ipython-input-3-02d234dfe405>:19: DeprecationWarning: use options instead of chrome_options
  with webdriver.Chrome(chrome_options=options) as driver:
<ipython-input-3-02d234dfe405>:20: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for link in tqdm.tqdm_notebook(sorted(df.link.unique())):


In [7]:
if os.path.exists('parsed_videos_info.json'):
    os.remove('parsed_videos_info.json')

with open('parsed_videos_info.json', 'a+') as output:
    output.write(json.dumps(resources))

In [8]:
df = pd.read_json('parsed_videos_info.json')
df

,link,title,canal_nome,canal_link,view_counts,video_date,like,dislike
0,https://www.youtube.com/watch?v=-0WQnwNFqJM,AI learns to Speedrun QWOP using Machine Learning,Wesley Liao,https://www.youtube.com/channel/UCair-JZ6XmtJk...,226613.0,25 de fev. de 2021,"2.729 marcações ""Gostei""","33 marcações ""Não gostei"""
1,https://www.youtube.com/watch?v=-5hEYRt8JE0,How to Become A Machine Learning Engineer | Ho...,#MachineLearningAlgorithms,https://www.youtube.com/hashtag/machinelearnin...,53263.0,3 de set. de 2018,"1.407 marcações ""Gostei""","27 marcações ""Não gostei"""
2,https://www.youtube.com/watch?v=-6RqxhNO2yY,Python For Data Science Full Course - 9 Hours ...,#edureka,https://www.youtube.com/hashtag/edureka,142715.0,15 de mar. de 2020,"3.990 marcações ""Gostei""","57 marcações ""Não gostei"""
3,https://www.youtube.com/watch?v=-AkBfBWr_Gw,Can You Become a Data Scientist?,#data,https://www.youtube.com/hashtag/data,776910.0,14 de ago. de 2018,"12.316 marcações ""Gostei""","431 marcações ""Não gostei"""
4,https://www.youtube.com/watch?v=-DDrE6T0ct4,Binary Image Classification with CNN (Deep Lea...,#download,https://www.youtube.com/hashtag/download,2290.0,23 de mar. de 2020,"66 marcações ""Gostei""","Sem marcações ""Não gostei"""
...,...,...,...,...,...,...,...,...
1763,https://www.youtube.com/watch?v=zSx7EBOI6xI,Kaggle Live-Coding: Code Reviews! Class imbala...,Kaggle,https://www.youtube.com/channel/UCSNeZleDn9c74...,4102.0,,"103 marcações ""Gostei""","2 marcações ""Não gostei"""
1764,https://www.youtube.com/watch?v=zUk3Dv-0xcc,【清华生学习日常】我用一个假期自学完Machine Learning！优秀的背后是夜以继日的刻苦,锦堂生活空间,https://www.youtube.com/channel/UC8_o9aFpknEMc...,80143.0,,"2.700 marcações ""Gostei""","48 marcações ""Não gostei"""
1765,https://www.youtube.com/watch?v=zX1RnFsJEPU,HIN Webcast: The Revolution of Tomorrow - Arti...,BOSTON,https://www.youtube.com/results?search_query=B...,527.0,12 de mar. de 2021,"11 marcações ""Gostei""","Sem marcações ""Não gostei"""
1766,https://www.youtube.com/watch?v=zZIksipqKpA,Week 1 of Kaggle BIPOC,MALAYSIA,https://www.youtube.com/results?search_query=M...,16.0,11 de mar. de 2021,"3 marcações ""Gostei""","Sem marcações ""Não gostei"""


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1768 entries, 0 to 1767
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   link         1768 non-null   object 
 1   title        1741 non-null   object 
 2   canal_nome   1748 non-null   object 
 3   canal_link   1748 non-null   object 
 4   view_counts  1747 non-null   float64
 5   video_date   1748 non-null   object 
 6   like         1711 non-null   object 
 7   dislike      1711 non-null   object 
dtypes: float64(1), object(7)
memory usage: 110.6+ KB


In [10]:
df.sample(25)

,link,title,canal_nome,canal_link,view_counts,video_date,like,dislike
1309,https://www.youtube.com/watch?v=ji7DtgUeKCI,7 Things to Look For in a Masters For Data Sci...,#DataScience,https://www.youtube.com/hashtag/datascience,9742.0,20 de nov. de 2020,"368 marcações ""Gostei""","7 marcações ""Não gostei"""
688,https://www.youtube.com/watch?v=NL3U8QOGBaE,Kaggle Live Coding: Brown Clustering | Kaggle,Kaggle,https://www.youtube.com/channel/UCSNeZleDn9c74...,2351.0,,"52 marcações ""Gostei""","Sem marcações ""Não gostei"""
1407,https://www.youtube.com/watch?v=n1ptHGG3dm4,Курсы по Data Science / Зарплаты / Поступать л...,#DataScience,https://www.youtube.com/hashtag/datascience,14067.0,,"802 marcações ""Gostei""","26 marcações ""Não gostei"""
1295,https://www.youtube.com/watch?v=ixhvCZNChpU,شرح موقع kaggle,Kareem Negm,https://www.youtube.com/channel/UCJmqZiCgUhNhE...,1221.0,Estreou em 21 de jun. de 2020,"29 marcações ""Gostei""","1 marcação ""Não gostei"""
261,https://www.youtube.com/watch?v=861NAO5-XJo,Kaggle Competition Pipeline Demo,LOS ANGELES,https://www.youtube.com/results?search_query=L...,2282.0,Estreou em 9 de fev. de 2020,"62 marcações ""Gostei""","7 marcações ""Não gostei"""
580,https://www.youtube.com/watch?v=Iuz_jc96bQk,O que é Machine Learning? #HipstersPontoTube,#HipstersPontoTube,https://www.youtube.com/hashtag/hipsterspontotube,73247.0,22 de abr. de 2019,"6.454 marcações ""Gostei""","71 marcações ""Não gostei"""
42,https://www.youtube.com/watch?v=0QYhaFyftLM,Data Scientist vs Data Analyst vs Data Enginee...,#DataScientistvsDataAnalystvsDataEngineer,https://www.youtube.com/hashtag/datascientistv...,86640.0,17 de jun. de 2019,"1.710 marcações ""Gostei""","58 marcações ""Não gostei"""
1607,https://www.youtube.com/watch?v=tk-vPly7vf4,Как искусственный интеллект и нейросети меняют...,#FutureToday,https://www.youtube.com/hashtag/futuretoday,49338.0,19 de fev. de 2021,"1.243 marcações ""Gostei""","38 marcações ""Não gostei"""
468,https://www.youtube.com/watch?v=FNfBR7ZnyWo,Top 5 Data Science Projects/Ideas For Your Res...,#datascienceprojects,https://www.youtube.com/hashtag/datasciencepro...,35778.0,,"1.815 marcações ""Gostei""","19 marcações ""Não gostei"""
1211,https://www.youtube.com/watch?v=g335THJxkto,Kaggle BNP Paribas — Станислав Семенов,ODS AI Ru,https://www.youtube.com/channel/UCeq6ZIlvC9SVs...,16429.0,2 de jun. de 2016,"275 marcações ""Gostei""","10 marcações ""Não gostei"""


In [2]:
df = pd.read_json('parsed_videos_info.json')

In [3]:
df.to_feather('raw_data.feather')
df.to_excel('raw_data_sem_label.xlsx', sheet_name='links')